In [1]:
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict

from data_prep_scripts.data_countries import get_country_of_origin_data 
from data_prep_scripts.data_manipulation import df_replaceColVals_vars,process_remaining_categ_cols
from data_prep_scripts.process_repetitive_cols import get_repetitive_cols

In [2]:
medData = pd.read_excel('Merged File 2.5.19 De-identified.xlsx')
medData['Country of Origin for Father'].replace(76,'Ireland',inplace=True)
medData = medData[(medData['Breast Cancer?']=='Yes') & (medData['Gender'] == 'Female')]
medData = medData.applymap(lambda s:s.lower() if type(s) == str else s)
predCol = pd.read_excel('Additional BCD Info 3.1.19 De-identified.xlsx',usecols="A,DZ")
medData = medData.merge(predCol,left_on='ID #', right_on='ID #')

In [3]:
ind_repititive_blocks = [
    (47,89,7),(89,117,7),(117,152,7),
    (163,191,7),(197,213,4),(214,249,5),
    (294,348,6),(349,356,7),(357,364,7),
    (427,435,8)
]
ind_not_del = list(range(22,43)) + [274]
ind_to_del = [2,3,4,(8,11),13,451,(197,213),(349,356),(448,469)]

In [4]:
# processes blocks of columns that contain the same type of information
# groups them and returns them in lists.
rep_cols = get_repetitive_cols(medData)

In [5]:
# get features for country of origin columns
country_of_origin_data = get_country_of_origin_data(medData)

In [6]:
# mixed_cols=[]
# for ii, (a,b) in enumerate(medData.dtypes.iteritems()):
#     if b == object:
#         types = set([type(k) for k in list(medData[a].dropna() ) ])
#         if str in types and len(types)>1: 
#             print(ii,a,b)

In [7]:
# columns that should only contain real numbered values, but contain strings in the raw data.
# this part of the code saves such string values in a json file, where we can specify
# the values to replace them with.
mixedCols_write_strs = [92,121,126,131,136,141,146,151,152,192,294,303,416,417,418,446,449]
dict_replaceColVals = df_replaceColVals_vars(medData,mixedCols_write_strs,str_vals=True)

In [8]:
medData.replace(dict_replaceColVals,inplace=True)

In [9]:
# replace outlier values with the values given in the json file
# where we can specify the values to replace them with
dict_replaceColVals_cont = df_replaceColVals_vars(medData,str_vals=False, cont_vals=True)
for k,v in dict_replaceColVals_cont.items():
    k_l, v_l = [], []
    for key, val in v.items():
        k_l.append(float(key)); v_l.append(val)
    medData[k].replace(k_l,v_l,inplace=True)
        

/home/serkan/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/serkan/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/serkan/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/serkan/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/serkan/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
col_min_max_count = 250
val_count_threshold = 100
col_values_replace_OTH = defaultdict(lambda:[])
cols_to_del=[]
for c in ind_to_del: 
    if isinstance(c,tuple): cols_to_del += list(medData.columns[c[0]:c[1]])
    else: cols_to_del.append(medData.columns[c])

for ii, (a,b) in enumerate(medData.dtypes.iteritems()):
    # do not delete the column in repetitive blocks
    in_rep_block = False
    for block in ind_repititive_blocks: 
        if (ii>=block[0]) and (ii < block[1]): in_rep_block = True
            
    if (not in_rep_block) and (b == object) and (ii not in ind_not_del):
        col_counts = medData[a].value_counts()        
        col_max_count = col_counts.iloc[0]
        # the most frequent value of a column should occur at least 'col_min_max_count' times
        if col_max_count < col_min_max_count: cols_to_del.append(a)
        elif medData.shape[0]-col_max_count < col_min_max_count: cols_to_del.append(a)
        # for the columns that we preserve, we replace values if their 
        # frequency is not above the given threshold 'val_count_threshold'.
        else:
            vals_to_OTH = list(col_counts[col_counts<val_count_threshold].index)
            if vals_to_OTH: 
                col_values_replace_OTH[a] = vals_to_OTH
# print(len(col_values_replace_OTH.keys()))
# print(col_values_replace_OTH.keys())
# print(len(cols_to_del))
# print(cols_to_del)

In [11]:
# I deleted the datetime values for now. Will be added in the next iterations.
cols_to_del += list(medData.select_dtypes(include=['datetime']))
# cols processed by the 'get_repetitive_cols()' function
cols_repetitive = sum([list(medData.columns[c[0]:c[1]]) for c in ind_repititive_blocks],[])
# column we predict
col_to_predict = 'Triple Negative'

In [12]:
remaining_obj_cols = [
    k for k in medData.select_dtypes(include=['object']) 
        if k not in cols_to_del + cols_repetitive +[col_to_predict]
]

In [13]:
#remaining_obj_cols

In [14]:
# for i,k in enumerate(medData.columns):
#     if k in remaining_obj_cols: print(i)